# Creating a new workspacwee using SDK
* Automatically creates a new workspace for us
* Remeber when you run the script below, there will be authentication steps
* Once workspace is created cpu, gpu ( compute target for training and predicting your models) will be made available

In [3]:
from azureml.core import Workspace

ws = Workspace.create(name='myworkspace2',
               subscription_id='ff711122-6294-4fad-9d1f-bf505a51fc42',
               resource_group='mlproject',
               create_resource_group=False,
               location='westus2'
               )

Deploying StorageAccount with name myworkspstorage73fec2c74.
Deploying AppInsights with name myworkspinsights0010dae8.
Deployed AppInsights with name myworkspinsights0010dae8. Took 2.85 seconds.
Deploying KeyVault with name myworkspkeyvault85469494.
Deployed KeyVault with name myworkspkeyvault85469494. Took 16.87 seconds.
Deployed StorageAccount with name myworkspstorage73fec2c74. Took 23.62 seconds.
Deploying Workspace with name myworkspace2.
Deployed Workspace with name myworkspace2. Took 18.58 seconds.


In [4]:
ws = Workspace.get("myworkspace2", subscription_id='ff711122-6294-4fad-9d1f-bf505a51fc42',
               resource_group='mlproject',
               location='westus2')

# Save Config File
To use the same workspace in multiple environments, create a JSON configuration file. The configuration file saves your subscription, resource, and workspace name so that it can be easily loaded. To save the configuration use the write_config method.

In [8]:
ws.write_config(path='C:/Users/User/Desktop', file_name="ws_config.json")

# Load the workspace from the saved config file

In [12]:
ws = Workspace.from_config('C:/Users/User/Desktop/.azureml/ws_config.json')
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.32.0 to work with myworkspace2


# Diabetes Model

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
dataframe=pd.read_csv('C:/Users/User/Desktop/Data/diabetes.csv')

In [79]:
cols=['Pregnancies', 'PlasmaGlucose', 'DiastolicBloodPressure',
       'TricepsThickness', 'SerumInsulin', 'BMI', 'DiabetesPedigree', 'Age',
       'Diabetic']

In [80]:
dataframe=dataframe[cols]
dataframe

,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age,Diabetic
0,0,171,80,34,23,43.509726,1.213191,21,0
1,8,92,93,47,36,21.240576,0.158365,23,0
2,7,115,47,52,35,41.511523,0.079019,23,0
3,9,103,78,25,304,29.582192,1.282870,43,1
4,1,85,59,27,35,42.604536,0.549542,22,0
...,...,...,...,...,...,...,...,...,...
9995,6,95,85,37,267,18.497542,0.660240,31,0
9996,0,55,51,7,50,21.865341,0.086589,34,0
9997,5,99,59,47,67,30.774018,2.301594,43,1
9998,0,145,67,30,21,18.811861,0.789572,26,0


In [81]:
X_df=dataframe[['Pregnancies', 'PlasmaGlucose', 'DiastolicBloodPressure','TricepsThickness', 'SerumInsulin', 'BMI', 'DiabetesPedigree', 'Age']]
Y_df=dataframe['Diabetic']

In [82]:
X=X_df.values
Y=Y_df.values

In [83]:
Y

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [84]:
scaler=MinMaxScaler(feature_range=(0,1))
rescaledX=scaler.fit_transform(X)

In [85]:
test_size=0.33
seed=7
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=test_size,random_state=seed)

In [90]:
model=LogisticRegression(max_iter=100000)
model.fit(X_train,Y_train)
result=model.score(X_test,Y_test)
result

0.8021212121212121

# Write Script into Prep.py
First, let's create a folder for the script files we'll use in the pipeline steps.

In [98]:
import os
# Create a folder for the pipeline step files
train_folder = 'train'
os.makedirs(train_folder, exist_ok=True)

print(train_folder)

train


In [ ]:
%%writefile $prep_folder/prep.py 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
dataframe=pd.read_csv('C:/Users/User/Desktop/Data/diabetes.csv')

cols=['Pregnancies', 'PlasmaGlucose', 'DiastolicBloodPressure',
       'TricepsThickness', 'SerumInsulin', 'BMI', 'DiabetesPedigree', 'Age',
       'Diabetic']

dataframe=dataframe[cols]


X_df=dataframe[['Pregnancies', 'PlasmaGlucose', 'DiastolicBloodPressure','TricepsThickness', 'SerumInsulin', 'BMI', 'DiabetesPedigree', 'Age']]
Y_df=dataframe['Diabetic']

X=X_df.values
Y=Y_df.values

scaler=MinMaxScaler(feature_range=(0,1))
rescaledX=scaler.fit_transform(X)

test_size=0.33
seed=7
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=test_size,random_state=seed)

import joblib
os.makedirs("outputs", exist_ok=True)
joblib.dump(value=price_room, filename="prep/bh_lr.pkl")


# Boston

In [96]:
%%writefile my_sklearn_lr.py
# Load the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sklearn
import joblib
import datetime
from azureml.core import Run
run = Run.get_context()
# Load Boston data
from sklearn.datasets import load_boston
boston_dataset = load_boston()
# Train test split data
from sklearn.model_selection import train_test_split
num_Rooms_Train, num_Rooms_Test, med_price_Train, med_Price_Test = train_test_split(boston_dataset.data[:,5].reshape(-1,1), boston_dataset.target.reshape(-1,1))
# Create Linear Regression model
from sklearn.linear_model import LinearRegression
price_room = LinearRegression()
price_room.fit (num_Rooms_Train,med_price_Train)

Overwriting my_sklearn_lr.py


In [95]:
# Load the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sklearn
import joblib
import datetime
from azureml.core import Run
run = Run.get_context()
run.log("Training start time", str(datetime.datetime.now()))
# Load Boston data
from sklearn.datasets import load_boston
boston_dataset = load_boston()
# Train test split data
from sklearn.model_selection import train_test_split
num_Rooms_Train, num_Rooms_Test, med_price_Train, med_Price_Test = train_test_split(boston_dataset.data[:,5].reshape(-1,1), boston_dataset.target.reshape(-1,1))
# Create Linear Regression model
from sklearn.linear_model import LinearRegression
price_room = LinearRegression()
result=price_room.fit (num_Rooms_Train,med_price_Train)
run.log('Score :', result)
run.log("Training end time", str(datetime.datetime.now()))
run.complete()

Attempted to log scalar metric Training start time:
2021-07-18 21:57:19.989181
Attempted to log scalar metric Score ::
LinearRegression()
Attempted to log scalar metric Training end time:
2021-07-18 21:57:20.055180


# Save the model
Now that you have a model named price_room, let’s save this model as a pickle file:

In [47]:
import joblib
os.makedirs("outputs", exist_ok=True)
joblib.dump(value=price_room, filename="outputs/bh_lr.pkl")

['outputs/bh_lr.pkl']

# Registering Model into Workspace
With your model saved as a pickle file, you can upload it into your workspace:

In [32]:
from azureml.core.model import Model
model = Model.register(workspace=ws, model_path="./outputs/bh_lr.pkl", model_name="boston_housing_lr")

Registering model boston_housing_lr


# Training a Model using Azure ML
What if your computer only has 2GB of RAM (I believe this is highly unlikely…)? You can train your model with resources provided by Azure ML Services instead of using your local resources. After importing the relevant libraries, you can take a look at all the virtual machines that are available for you:

In [44]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.compute import AmlCompute
list_vms = AmlCompute.supported_vmsizes(workspace=ws)
# print(list_vms)
# print(type(list_vms))

In [27]:
import pandas as pd
pd.DataFrame(list_vms)

,name,vCPUs,gpus,memoryGB,maxResourceVolumeMB
0,Standard_E2a_v4,2,0,16.0,51200
1,Standard_E4a_v4,4,0,32.0,102400
2,Standard_E8a_v4,8,0,64.0,204800
3,Standard_E16a_v4,16,0,128.0,409600
4,Standard_E32a_v4,32,0,256.0,819200
...,...,...,...,...,...
108,Standard_ND24rs,24,4,448.0,1376256
109,Standard_ND24s,24,4,448.0,1376256
110,Standard_NV12s_v3,12,1,112.0,344064
111,Standard_NV24s_v3,24,2,224.0,688128


# Dependencies
For your model to work in the cloud, you need to specify the dependencies. Treat dependencies as a grocery shopping list. You tell the program what kind of grocery/libraries you need for you to cook a meal/train your model.

In [5]:
from azureml.core import *
import azureml.dataprep as dprep


# Create a Pipeline¶
You can perform the various steps required to ingest data, train a model, and register the model individually by using the Azure ML SDK to run script-based experiments. However, in an enterprise environment it is common to encapsulate the sequence of discrete steps required to build a machine learning solution into a pipeline that can be run on one or more compute targets; either on-demand by a user, from an automated build process, or on a schedule.

In this notebook, you'll bring together all of these elements to create a simple pipeline that pre-processes data and then trains and registers a model.

# Connect to your workspace
To get started, connect to your workspace.

Note: If you haven't already established an authenticated session with your Azure subscription, you'll be prompted to authenticate by clicking a link, entering an authentication code, and signing into Azure.

In [34]:
ws = Workspace.get("myworkspace2", subscription_id='ff711122-6294-4fad-9d1f-bf505a51fc42',
               resource_group='mlproject',
               location='westus2')
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.32.0 to work with myworkspace2


Start by creating a new Jupyter Notebook and follow the below steps. Run each of these blocks in a separate Notebook cell.

In [37]:
import os
import azureml.core
from azureml.core import Workspace, Experiment, Datastore
from azureml.widgets import RunDetails
 
from azureml.core import Dataset
 
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.core import PipelineRun, StepRun, PortDataReference
from azureml.pipeline.steps import PythonScriptStep
 
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
 
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
 
from azureml.core.model import Model
 
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.32.0


# Prepare the Data
In your pipeline, you'll use a dataset containing details of diabetes patients. Run the cell below to create this dataset (if you created it previously, the code will find the existing version)